<a href="https://colab.research.google.com/github/agatagruza/private-ai/blob/master/SPAIC_Project9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project 9:  Federated Learning 

***Federated Learning is a technique for training Deep Learning models on data to which you do not have access.***

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

- app company (Texting prediction app)
- predictive maintenance (automobiles / industrial engines)
- wearable medical devices
- ad blockers / autotomplete in browsers (Firefox/Brave)

Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

- privacy concerns: legal, user discomfort, competitive dynamics
- engineering: the bandwidth/storage requirements of aggregating the larger dataset


##Basic Remote Execution in PySyft

**The essence of Federated Learning is the ability to train models in parallel on a wide number of machines.** Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors use **pointers** 

In [0]:
pip install syft

In [0]:
import torch as th
import syft as sy

In [0]:
hook = sy.TorchHook(th)

In [0]:
bob = sy.VirtualWorker(hook, id = "bob")  

In [46]:
bob._objects

{}

In [0]:
x = th.tensor([1, 2, 3, 4, 5]) 
x = x.send(bob)

In [48]:
bob._objects 

{92453301336: tensor([1, 2, 3, 4, 5])}

In [49]:
x.location

<VirtualWorker id:bob #objects:1>

In [50]:
x.id_at_location

92453301336

In [51]:
x.id

6298448583

In [52]:
x.owner

<VirtualWorker id:me #objects:0>

In [57]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [60]:
bob._objects 

{}

In [0]:
<TO DO>

#Playing with Remote Tensors
In this project, .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [0]:
bob = sy.VirtualWorker(hook, id = "bob") 
alice = sy.VirtualWorker(hook, id = "alice")
x = th.tensor([1, 2, 3, 4, 5])

In [0]:
x = x.send(bob, alice) 

In [63]:
x 

(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:70106807806 -> bob:96654452005]
	-> (Wrapper)>[PointerTensor | me:7734596226 -> alice:53580796097]

In [64]:
bob._objects 

{96654452005: tensor([1, 2, 3, 4, 5])}

In [65]:
alice._objects

{53580796097: tensor([1, 2, 3, 4, 5])}

In [66]:
x.get()

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [67]:
bob._objects 

{}

In [68]:
alice._objects 

{}

In [0]:
x = th.tensor([1, 2, 3, 4, 5]).send(bob, alice)

In [0]:
z = x.get(sum_results = True)

In [71]:
z

tensor([ 2,  4,  6,  8, 10])

#Introducing Remote Arithmetic

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob) 
y = th.tensor([1,1,1,1,1]).send(bob)

In [73]:
x

(Wrapper)>[PointerTensor | me:91518782923 -> bob:42405821493]

In [74]:
y

(Wrapper)>[PointerTensor | me:6862419777 -> bob:55902714851]

In [0]:
z = x + y

In [76]:
z

(Wrapper)>[PointerTensor | me:32719706749 -> bob:37360888294]

In [77]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [78]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:40165670998 -> bob:16182602655]

In [79]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [0]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [0]:
z = (x + y).sum()

In [82]:
z.backward()

(Wrapper)>[PointerTensor | me:38964464159 -> bob:60582668156]

In [0]:
x = x.get()

In [84]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [85]:
x.grad

tensor([1., 1., 1., 1., 1.])